In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor

from functions.datacleaning import MoscowHousing as mh
from functions.distance import get_distance_coordinates, norm_features

import matplotlib.pylab as plt
import seaborn as sns
plt.style.use('ggplot')

# Prepare NaN Values

In [2]:
# Get data
data = mh()

XTrain, YTrain = data.get_data_train()
XTest = data.get_data_test()

In [3]:
# Template for replacing nan by computing other variables
#df["parking"] = df.groupby("district").transform(lambda x: x.fillna(x.median()))["parking"]

In [4]:
#df.column_name.loc[df.column_name > max_value]  = max_value

In [5]:
district_counts = {0:{},1:{},2:{},3:{},4:{},5:{},6:{},7:{},8:{},9:{},10:{},11:{},12:{}}
districts = XTrain["district"]
for d in districts:
    pass